In [614]:
# Efetivo(interno)
# Jornada > Carga hora
# Area de conhecimento
# Segunda habilitação
# Preferencia
# 
# 
# Vagas sem preenchimento
# Substituição temporario ~~ Efetivo
# 
# Turno

In [615]:
def desempate():
    for objeto_1, escola_1 in escolas_dict.items():
        if escola_1.lista_professores:
            professor = escola_1.lista_professores[0]
            carencia = escola_1.VAGAS - len(escola_1.lista_professores)

            # for index, prof in base_professores.iterrows():
            #     prof_turno = prof['TURNO_CONCENTRADO']

            
            for objeto_verificado, escola_verificada in escolas_dict.items():
                if objeto_verificado != objeto_1:  # Garante que não estamos verificando a mesma escola_1

                    if professor in escola_verificada.lista_professores:
                        indice = escola_verificada.lista_professores.index(professor)
                        carencia_verificada = escola_verificada.VAGAS - len(escola_verificada.lista_professores)
                        top3 = 0, 1, 2

                        if escola_1.VAGAS == 0 and escola_1.lista_professores:
                            escola_1.lista_professores.pop(0)
                            break
                        elif escola_verificada.VAGAS == 0 and escola_verificada.lista_professores:
                            escola_verificada.lista_professores.pop(0)
                            break
                        else:

                            if indice + 1 > escola_verificada.VAGAS: # Garante que o professore podera ser lotado em ambas as escolas
                                pass

                            elif indice + 1 <= escola_verificada.VAGAS:

                                    preferencias = []
                                    for index, servidor in base_professores.iterrows():
                                        if professor == servidor['SERVIDOR']:
                                            preferencias = [base_professores.at[index, f'PREFERENCIA {i}'] for i in range(1, 6)]
                                            break
                                    try:
                                        escola1 = preferencias.index(escola_1.COD_ESCOLA)
                                    except ValueError:
                                        escola1 = -1
                                    try:
                                        escola2 = preferencias.index(escola_verificada.COD_ESCOLA)
                                    except ValueError:
                                        escola2 = -1

                                    if indice > 0: # TOP 3 PREFERENCIAS
                                        if carencia_verificada > -1 and escola2 in top3:
                                            escola_1.lista_professores.pop(0)
                                            print(professor)
                                        else:
                                            escola_verificada.lista_professores.pop(indice)
                                            break
                                        
                                    else:

                                        if escola1 >= 0 and escola2 < 0:                    #  Professor sem preferencia pela Escola Verificada
                                            escola_verificada.lista_professores.pop(0)
                                            break

                                        elif escola2 >= 0 and escola1 < 0:                  #  Professor sem preferencia pela Escola
                                            escola_1.lista_professores.pop(0)
                                            break

                                        elif escola1 >= 0 and escola2 >= 0:                 #  Professor com preferencia em ambas
                                            if escola1 < escola2:                           # Verificando qual a primeira prefencia
                                                escola_verificada.lista_professores.pop(0)
                                                break
                                            elif escola1 > escola2: 
                                                escola_1.lista_professores.pop(0)
                                                break
                                            else:
                                                if carencia_verificada == 0:
                                                    if carencia > 0:
                                                        escola_verificada.lista_professores.pop(0)
                                                        break
                                                    elif carencia < 0:
                                                        escola_1.lista_professores.pop(0)
                                                        break
                                                    else:
                                                        escola_verificada.lista_professores.pop(0)  # A DENFINIR MELHOR CRITE
                                                        break
                                                else:
                                                    if carencia_verificada > 0:
                                                        escola_1.lista_professores.pop(0)
                                                        break
                                                    elif carencia_verificada < 0:
                                                        escola_verificada.lista_professores.pop(0)
                                                        break
                                        
                                        elif escola1 < 0 and escola2 < 0:                   #  Professor sem preferencia em ambas
                                                # Verificando maior carencia
                                            if carencia_verificada == 0:
                                                if carencia > 0:
                                                    escola_verificada.lista_professores.pop(0)
                                                    break
                                                elif carencia < 0:
                                                    escola_1.lista_professores.pop(0)
                                                    break
                                                else:
                                                    escola_verificada.lista_professores.pop(0)  # A DENFINIR MELHOR CRITE
                                                    break
                                            else:
                                                if carencia_verificada > 0:
                                                    escola_1.lista_professores.pop(0)
                                                    break
                                                elif carencia_verificada < 0:
                                                    escola_verificada.lista_professores.pop(0)
                                                    break


def lotacao(dataFrame):
    df_atual = dataFrame
    global professores_lotados
    while True:
        # Mantém a flag para verificar se houve alocação em algum loop
        alocou = False

        desempate()

        for nome_objeto, escola in escolas_dict.items():
            # Verifica se a lista de professores não está vazia
            if escola.VAGAS > 0 and escola.lista_professores:
                # Obtendo o primeiro professor da lista
                professor = escola.lista_professores[0]
                for index, prof in base_professores.iterrows():
                    if prof['SERVIDOR'] == professor:
                        servidor = prof['SERVIDOR']
                        jornada = prof['JORNADA']
                        efetivo = prof['EFETIVO']

                # Criando um dicionário com os dados da escola e do professor
                dados = {
                    'URE': escola.URE,
                    'USE': escola.USE,
                    'MUNICIPIO': escola.MUNICIPIO,
                    'AGRUPAMENTO': escola.AGRUPAMENTO,
                    'COD_ESCOLA': escola.COD_ESCOLA,
                    'COD_MEC': escola.COD_MEC,
                    'COD_SETOR': escola.COD_SETOR,
                    'ESCOLA': escola.ESCOLA,
                    'COD_DISCIPLINA': escola.COD_DISCIPLINA,
                    'CH_HORARIA': escola.CH_HORARIA,
                    'SERVIDOR': professor,  # Adiciona um professor por vez
                    'EFETIVO': efetivo,
                    'JORNADA': jornada
                }

                # Adicionando os dados do professor ao DataFrame professores_lotados
                professores_lotados = professores_lotados._append(dados, ignore_index=True)

                escola.CH_RELATIVA -= escola.CH_HORARIA

                # Removendo o professor alocado da lista de professores
                escola.lista_professores.pop(0)

                # Verificações e ações adicionais dentro do while

                for objeto_verificado, escola_verificada in escolas_dict.items():
                    if objeto_verificado != nome_objeto:  # Garante que não estamos verificando a mesma escola
                        if professor in escola_verificada.lista_professores:
                            indice = escola_verificada.lista_professores.index(professor)
                            escola_verificada.lista_professores.pop(indice)

                # Subtraindo 1 da quantidade de vagas
                escola.VAGAS -= 1

                # Excluindo o professor alocado do DataFrame base_professores
                for index, servidor in df_atual.iterrows():
                    if professor == servidor['SERVIDOR']:
                        base_professores.drop(index, inplace=True)
                        break  # Interrompe o loop depois de encontrar e remover o professor

                # Marca que houve alocação nesta iteração do loop
                alocou = True

        # Se nenhum professor foi alocado em nenhum loop, encerra o processo
        if not alocou:
            for nome_objeto, escola in escolas_dict.items():
                escola.lista_professores.clear()
            break




In [616]:
import pandas as pd
import math 

class Escola:
    def __init__(self, URE, USE, MUNICIPIO, AGRUPAMENTO, COD_ESCOLA, COD_MEC, COD_SETOR, ESCOLA, COD_DISCIPLINA, INTEGRAL, CH_HORARIA, VAGAS, lista_professores, CH_RELATIVA=""):
        self.URE = URE
        self.USE = USE
        self.MUNICIPIO = MUNICIPIO
        self.AGRUPAMENTO = AGRUPAMENTO
        self.COD_ESCOLA = COD_ESCOLA
        self.COD_MEC = COD_MEC
        self.COD_SETOR = COD_SETOR
        self.ESCOLA = ESCOLA
        self.COD_DISCIPLINA = COD_DISCIPLINA
        self.INTEGRAL = INTEGRAL
        self.CH_HORARIA = CH_HORARIA
        self.VAGAS = VAGAS
        self.lista_professores = lista_professores
        self.CH_RELATIVA = CH_RELATIVA

# Carregar o DataFrame
base_escolas = pd.read_excel('escolas.xlsx')

# Dicionário para armazenar os objetos Escola
escolas_dict = {}

# Percorrer as linhas do DataFrame
for indice, linha in base_escolas.iterrows():
    # Gerar o nome do objeto
    nome_objeto = f"escola_{linha['COD_ESCOLA']}_{linha['COD_DISCIPLINA']}_{linha['CH_HORARIA']}"
    
    # Calcular CH_RELATIVA
    ch_relativa = linha['CH_HORARIA'] * linha['VAGAS']

    vagas = math.ceil(ch_relativa / linha['CH_HORARIA'])
    
    # Criar um objeto Escola para cada linha com CH_RELATIVA e VAGAS calculadas
    escola = Escola(
        linha['URE'],
        linha['USE'],
        linha['MUNICIPIO'],
        linha['AGRUPAMENTO'],
        linha['COD_ESCOLA'],
        linha['COD_MEC'],
        linha['COD_SETOR'],
        linha['ESCOLA'],
        linha['COD_DISCIPLINA'],
        linha['INTEGRAL'],
        linha['CH_HORARIA'],
        vagas,  # Definir VAGAS com o valor arredondado para cima
        lista_professores=[],
        CH_RELATIVA=ch_relativa  # Definir CH_RELATIVA como o produto entre CH_HORARIA e VAGAS
    )

    # Adicionar o objeto ao dicionário usando o nome como chave
    escolas_dict[nome_objeto] = escola

for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, VAGAS: {escola_print.VAGAS} , CH em aberto {escola_print.CH_RELATIVA}")


# CONTRATAÇÃO POR PREFERENCIA
# SUBTRAI NUMERO DE VAGAS

# CONFERIR REPETIÇÃO NO PRIMEIRO ELEMENTO DAS OUTRAS LISTAS
# REMOÇÃO DO PROFESSOR DE OUTRAS LISTAS



Nome do objeto: escola_1_1_40, VAGAS: 3 , CH em aberto 120
Nome do objeto: escola_1_2002_40, VAGAS: 3 , CH em aberto 120
Nome do objeto: escola_1_2003_20, VAGAS: 1 , CH em aberto 20
Nome do objeto: escola_2_1_40, VAGAS: 1 , CH em aberto 40
Nome do objeto: escola_2_2002_40, VAGAS: 2 , CH em aberto 80
Nome do objeto: escola_2_2003_40, VAGAS: 5 , CH em aberto 200
Nome do objeto: escola_3_1_40, VAGAS: 1 , CH em aberto 40
Nome do objeto: escola_3_2002_20, VAGAS: 1 , CH em aberto 20
Nome do objeto: escola_3_2003_20, VAGAS: 1 , CH em aberto 20
Nome do objeto: escola_4_1_40, VAGAS: 1 , CH em aberto 40
Nome do objeto: escola_4_2002_20, VAGAS: 1 , CH em aberto 20
Nome do objeto: escola_4_2003_40, VAGAS: 1 , CH em aberto 40
Nome do objeto: escola_5_1_40, VAGAS: 2 , CH em aberto 80
Nome do objeto: escola_5_2002_40, VAGAS: 1 , CH em aberto 40
Nome do objeto: escola_5_2003_40, VAGAS: 1 , CH em aberto 40
Nome do objeto: escola_6_1_40, VAGAS: 1 , CH em aberto 40


In [617]:
base_professores = pd.read_excel('prof.xlsx')

# base_professores = base_professores.sort_values(by=['DATA_EXERCICIO', 'SEGUNDA_HABILITACAO', 'area_conhecimento', 'INTERNO', 'EFETIVO'], ascending=[True, False, False, False, False])

base_professores = base_professores.sort_values(by='DATA_EXERCICIO', ascending=True)
base_professores = base_professores.sort_values(by='SEGUNDA_HABILITACAO', ascending=False)
base_professores = base_professores.sort_values(by='area_conhecimento', ascending=False)
base_professores = base_professores.sort_values(by='EFETIVO', ascending=False)

professores_internos = base_professores[base_professores['INTERNO'] == 'SIM']

professores_internos = professores_internos.sort_values(by=['DATA_EXERCICIO', 'SEGUNDA_HABILITACAO', 'area_conhecimento', 'DT_EXERCICIO_NA_ESCOLA', 'CH_TOTAL', 'EFETIVO'], ascending=[True, False, False, True, False, False])

display(professores_internos)

global professores_lotados

professores_lotados = pd.DataFrame(columns=['URE', 'USE', 'MUNICIPIO', 'AGRUPAMENTO', 'COD_ESCOLA', 'COD_MEC','COD_SETOR', 'ESCOLA','COD_DISCIPLINA'])


,URE,USE,MUNICIPIO,AGRUPAMENTO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,SERVIDOR,...,CH_SEMANAL,CH_TOTAL,area_conhecimento,TURNO_N_APTO,TURNO_CONCENTRADO,PREFERENCIA 1,PREFERENCIA 2,PREFERENCIA 3,PREFERENCIA 4,PREFERENCIA 5
0,01A URE - BRAGANCA,NaN,BRAGANCA,A,1,15055582,1920910000000,EE ANDRE ALVES,2002,Professor 1,...,4,44,SIM,NAO,M,1,2,3,NaN,NaN
10,01A URE - BRAGANCA,NaN,BRAGANCA,A,1,15055582,1920910000000,EE ANDRE ALVES,2003,Professor 11,...,4,44,SIM,N,M,1,2,3,NaN,NaN
20,01A URE - BRAGANCA,NaN,BRAGANCA,A,1,15055582,1920910000000,EE ANDRE ALVES,1,Professor 21,...,4,44,SIM,NAO,M,1,2,3,NaN,NaN
3,01A URE - BRAGANCA,NaN,BRAGANCA,A,4,15057291,1920913530000,EEEF PATALINO,2002,Professor 4,...,3,39,NAO,NAO,T,4,3,2,NaN,NaN
13,01A URE - BRAGANCA,NaN,BRAGANCA,A,4,15057291,1920913530000,EEEF PATALINO,2003,Professor 14,...,3,39,NAO,NAO,T,4,3,2,NaN,NaN
23,01A URE - BRAGANCA,NaN,BRAGANCA,A,4,15057291,1920913530000,EEEF PATALINO,1,Professor 24,...,3,39,NAO,NAO,T,4,3,2,NaN,NaN
31,01A URE - BRAGANCA,NaN,BRAGANCA,B,6,15055582,1920910000000,ESCOLA X,1,Professor 98,...,3,39,SIM,NAO,N,6,4,3,2.0,1.0
22,01A URE - BRAGANCA,NaN,BRAGANCA,A,5,15057046,1920914340000,EEEFM DO ROCHA,1,Professor 23,...,4,40,SIM,NAO,N,3,1,2,NaN,NaN
12,01A URE - BRAGANCA,NaN,BRAGANCA,A,5,15057046,1920914340000,EEEFM DO ROCHA,2003,Professor 13,...,4,40,SIM,T,N,3,1,2,NaN,NaN
2,01A URE - BRAGANCA,NaN,BRAGANCA,A,5,15057046,1920914340000,EEEFM DO ROCHA,2002,Professor 3,...,4,40,SIM,NAO,M,3,1,2,NaN,NaN


In [618]:
# RODANDO APENAS PROFESSORES INTERNOS
for nome_objeto, escola in escolas_dict.items():

    for index, professor in professores_internos.iterrows():
        servidor = professor['SERVIDOR']
        
        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA']) and (escola.COD_ESCOLA == professor['COD_ESCOLA']) and (escola.AGRUPAMENTO == professor['AGRUPAMENTO']) and (professor['EFETIVO'] == 'SIM'):
            if professor['JORNADA'] != escola.CH_HORARIA:
                pass
            else:

                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)


print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)


lotacao(professores_internos) # Passar o DF de professores a ser usado



--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: ['Professor 21'], VAGAS: 3
Nome do objeto: escola_1_2002_40, PROFESSORES: ['Professor 1'], VAGAS: 3
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_1_40, PROFESSORES: ['Professor 22'], VAGAS: 1
Nome do objeto: escola_2_2002_40, PROFESSORES: ['Professor 2'], VAGAS: 2
Nome do objeto: escola_2_2003_40, PROFESSORES: ['Professor 12'], VAGAS: 5
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: ['Professor 4'], VAGAS: 1
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_5_1_40, PROFESSORES: ['Professor 23'], VAGAS: 2
Nome do objeto: escola_5_2002_40, PROFESSORES: ['Professor 3'], VAGAS: 1
Nome do objeto: escola_5_2003_40, PROFESSORES:

In [619]:
print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 4
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_5_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_5_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_6_1_40, PROFESSORES: [], VAGAS: 0
--------------------


In [620]:

# CRIANDO AS LISTAS PARA LOTAÇÃO INTEGRAL

for nome_objeto, escola in escolas_dict.items():

    if escola.INTEGRAL == 'SIM':
    
        for index, professor in base_professores.iterrows():
            pref = [base_professores.at[index, f'PREFERENCIA {i}'] for i in range(1, 6)]
            if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA'])and (escola.AGRUPAMENTO == professor['AGRUPAMENTO']) and (professor['EFETIVO'] == 'SIM'):
                if escola.COD_ESCOLA in pref:
                    servidor = professor['SERVIDOR']
                    
                    if servidor not in escola.lista_professores:
                        escola.lista_professores.append(servidor)

            else:
                pass


print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

lotacao(base_professores) # DataFrame usado


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 4
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_5_1_40, PROFESSORES: ['Professor 25', 'Professor 28'], VAGAS: 1
Nome do objeto: escola_5_2002_40, PROFESSORES: ['Professor 8', 'Professor 5'], VAGAS: 0
Nome do objeto: escola_5_2003_40, PROFESSORES: ['Professor 18', 'Professor 15'], VAGAS: 0
Nome d

In [621]:
print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 4
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_5_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_6_1_40, PROFESSORES: [], VAGAS: 0
--------------------


In [622]:

# CRIANDO AS LISTAS PARA LOTAÇÃO GERAL
for nome_objeto, escola in escolas_dict.items():

    for index, professor in base_professores.iterrows():

        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA'])and (escola.AGRUPAMENTO == professor['AGRUPAMENTO']) and (professor['EFETIVO'] == 'SIM'):
            if professor['JORNADA'] != escola.CH_HORARIA:
                pass
            else:
                servidor = professor['SERVIDOR']
                
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)

        else:
            pass


print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

lotacao(base_professores) # DataFrame usado


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: ['Professor 28'], VAGAS: 2
Nome do objeto: escola_1_2002_40, PROFESSORES: ['Professor 8', 'Professor 5'], VAGAS: 2
Nome do objeto: escola_1_2003_20, PROFESSORES: ['Professor 14', 'Professor 16', 'Professor 17'], VAGAS: 1
Nome do objeto: escola_2_1_40, PROFESSORES: ['Professor 28'], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: ['Professor 8', 'Professor 5'], VAGAS: 1
Nome do objeto: escola_2_2003_40, PROFESSORES: ['Professor 18', 'Professor 11', 'Professor 15'], VAGAS: 4
Nome do objeto: escola_3_1_40, PROFESSORES: ['Professor 28'], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: ['Professor 6', 'Professor 7'], VAGAS: 1
Nome do objeto: escola_3_2003_20, PROFESSORES: ['Professor 14', 'Professor 16', 'Professor 17'], VAGAS: 1
Nome do objeto: escola_4_1_40, PROFESSORES: ['Professor 28'], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: ['Professor 6', 'Professor 7'], VAGAS: 0
Nome do objeto: escola_4_

In [623]:
print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_6_1_40, PROFESSORES: [], VAGAS: 0
--------------------


In [624]:

# CRIANDO AS LISTAS PARA LOTAÇÃO FORA DO SUBGRUPO
for nome_objeto, escola in escolas_dict.items():

    for index, professor in base_professores.iterrows():

        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA'])and (escola.MUNICIPIO == professor['MUNICIPIO']) and (professor['EFETIVO'] == 'SIM'):
            if professor['JORNADA'] != escola.CH_HORARIA:
                pass
            else:
                servidor = professor['SERVIDOR']
                
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)

        else:
            pass


print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

lotacao(base_professores) # DataFrame usado


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: ['Professor 99'], VAGAS: 1
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_1_2003_20, PROFESSORES: ['Professor 17'], VAGAS: 0
Nome do objeto: escola_2_1_40, PROFESSORES: ['Professor 99'], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_3_1_40, PROFESSORES: ['Professor 99'], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: ['Professor 7'], VAGAS: 0
Nome do objeto: escola_3_2003_20, PROFESSORES: ['Professor 17'], VAGAS: 0
Nome do objeto: escola_4_1_40, PROFESSORES: ['Professor 99'], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: ['Professor 7'], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_1_40, PROFESSORES: ['Professor 99'], VAGAS: 0
Nome do objeto: escola_5_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2003_

In [625]:
professores_internos = base_professores[base_professores['INTERNO'] == 'SIM']

display(professores_internos)
# RODANDO APENAS PROFESSORES INTERNOS
for nome_objeto, escola in escolas_dict.items():

    for index, professor in professores_internos.iterrows():
        servidor = professor['SERVIDOR']
        
        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA']) and (escola.COD_ESCOLA == professor['COD_ESCOLA']) and (escola.AGRUPAMENTO == professor['AGRUPAMENTO']) and (professor['EFETIVO'] == 'SIM'):
            
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)

for nome_objeto, escola in escolas_dict.items():

    for index, professor in base_professores.iterrows():
        servidor = professor['SERVIDOR']
        
        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA']) and (escola.AGRUPAMENTO == professor['AGRUPAMENTO']) and (professor['EFETIVO'] == 'SIM'):
            
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)


print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

,URE,USE,MUNICIPIO,AGRUPAMENTO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,SERVIDOR,...,CH_SEMANAL,CH_TOTAL,area_conhecimento,TURNO_N_APTO,TURNO_CONCENTRADO,PREFERENCIA 1,PREFERENCIA 2,PREFERENCIA 3,PREFERENCIA 4,PREFERENCIA 5
23,01A URE - BRAGANCA,NaN,BRAGANCA,A,4,15057291,1920913530000,EEEF PATALINO,1,Professor 24,...,3,39,NAO,NAO,T,4,3,2,NaN,NaN


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: ['Professor 26', 'Professor 24', 'Professor 27'], VAGAS: 0
Nome do objeto: escola_1_2002_40, PROFESSORES: ['Professor 7'], VAGAS: 1
Nome do objeto: escola_1_2003_20, PROFESSORES: ['Professor 17'], VAGAS: 0
Nome do objeto: escola_2_1_40, PROFESSORES: ['Professor 26', 'Professor 24', 'Professor 27'], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: ['Professor 7'], VAGAS: 0
Nome do objeto: escola_2_2003_40, PROFESSORES: ['Professor 17'], VAGAS: 2
Nome do objeto: escola_3_1_40, PROFESSORES: ['Professor 26', 'Professor 24', 'Professor 27'], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: ['Professor 7'], VAGAS: 0
Nome do objeto: escola_3_2003_20, PROFESSORES: ['Professor 17'], VAGAS: 0
Nome do objeto: escola_4_1_40, PROFESSORES: ['Professor 24', 'Professor 26', 'Professor 27'], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: ['Professor 7'], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: ['Prof

In [626]:

while True:
        # Mantém a flag para verificar se houve alocação em algum loop
        alocou = False

        desempate()

        for nome_objeto, escola in escolas_dict.items():
            # Verifica se a lista de professores não está vazia
            if escola.VAGAS > 0 and escola.lista_professores:
                # Obtendo o primeiro professor da lista
                professor = escola.lista_professores[0]
                for index, prof in base_professores.iterrows():
                    if prof['SERVIDOR'] == professor:
                        servidor = prof['SERVIDOR']
                        jornada = prof['JORNADA']
                        efetivo = prof['EFETIVO']

                # Criando um dicionário com os dados da escola e do professor
                dados = {
                    'URE': escola.URE,
                    'USE': escola.USE,
                    'MUNICIPIO': escola.MUNICIPIO,
                    'AGRUPAMENTO': escola.AGRUPAMENTO,
                    'COD_ESCOLA': escola.COD_ESCOLA,
                    'COD_MEC': escola.COD_MEC,
                    'COD_SETOR': escola.COD_SETOR,
                    'ESCOLA': escola.ESCOLA,
                    'COD_DISCIPLINA': escola.COD_DISCIPLINA,
                    'CH_HORARIA': escola.CH_HORARIA,
                    'SERVIDOR': professor,  # Adiciona um professor por vez
                    'EFETIVO': efetivo,
                    'JORNADA': jornada
                }

                if escola.CH_RELATIVA > 0:
                    if escola.CH_HORARIA > jornada:
                        # Removendo o professor alocado da lista de professores
                        escola.lista_professores.pop(0)

                        # Verificações e ações adicionais dentro do while

                        for objeto_verificado, escola_verificada in escolas_dict.items():
                            if objeto_verificado != nome_objeto:  # Garante que não estamos verificando a mesma escola
                                if professor in escola_verificada.lista_professores:
                                    indice = escola_verificada.lista_professores.index(professor)
                                    escola_verificada.lista_professores.pop(indice)

                        # Excluindo o professor alocado do DataFrame base_professores
                        for index, servidor in base_professores.iterrows():
                            if professor == servidor['SERVIDOR']:
                                ch = escola.CH_RELATIVA - jornada # Subtraindo a Jornada do professor da CH da disciplina
                                escola.CH_RELATIVA = ch
                                #dados['CH_RELATIVA'] = ch
                                base_professores.drop(index, inplace=True)
                                break  # Interrompe o loop depois de encontrar e remover o professor

                        # Adicionando os dados do professor ao DataFrame professores_lotados
                        professores_lotados = professores_lotados._append(dados, ignore_index=True)
                    
                    elif escola.CH_HORARIA == jornada:
                        # Adicionando os dados do professor ao DataFrame professores_lotados
                        professores_lotados = professores_lotados._append(dados, ignore_index=True)
                        
                        escola.CH_RELATIVA -= escola.CH_HORARIA
                        # Removendo o professor alocado da lista de professores
                        escola.lista_professores.pop(0)

                        # Verificações e ações adicionais dentro do while

                        for objeto_verificado, escola_verificada in escolas_dict.items():
                            if objeto_verificado != nome_objeto:  # Garante que não estamos verificando a mesma escola
                                if professor in escola_verificada.lista_professores:
                                    indice = escola_verificada.lista_professores.index(professor)
                                    escola_verificada.lista_professores.pop(indice)

                        # Subtraindo 1 da quantidade de vagas
                        escola.CH_RELATIVA -= escola.CH_HORARIA

                        # Excluindo o professor alocado do DataFrame base_professores
                        for index, servidor in base_professores.iterrows():
                            if professor == servidor['SERVIDOR']:
                                base_professores.drop(index, inplace=True)
                                break  # Interrompe o loop depois de encontrar e remover o professor

                    else:
                        # Adicionando os dados do professor ao DataFrame professores_lotados
                        professores_lotados = professores_lotados._append(dados, ignore_index=True)

                        # Removendo o professor alocado da lista de professores
                        escola.lista_professores.pop(0)

                        # Verificações e ações adicionais dentro do while
                        CH = base_professores.at[index, 'JORNADA'] - escola.CH_HORARIA 
                        # Subtraindo 1 da quantidade de vagas
                        escola.CH_RELATIVA -= CH

                        # Excluindo o professor alocado do DataFrame base_professores
                        for index, servidor in base_professores.iterrows():
                            if professor == servidor['SERVIDOR']:
                                base_professores.drop(index, inplace=True)
                                base_professores.at[index, 'JORNADA'] == CH # Subtraindo a Jornada do professor da CH da disciplina
                                break  # Interrompe o loop depois de encontrar e remover o professor

                # Marca que houve alocação nesta iteração do loop
                alocou = True

        # Se nenhum professor foi alocado em nenhum loop, encerra o processo
        if not alocou:
            for nome_objeto, escola in escolas_dict.items():
                escola.lista_professores.clear()
            break



In [627]:
print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_4_2002_20, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_1_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2002_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_5_2003_40, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_6_1_40, PROFESSORES: [], VAGAS: 0
--------------------


In [628]:
for nome_objeto, escola in escolas_dict.items():
    escola.VAGAS = math.ceil(escola.CH_RELATIVA / escola.CH_HORARIA)

In [629]:
print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_1_2002_40, PROFESSORES: [], VAGAS: 0.5
Nome do objeto: escola_1_2003_20, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_2_1_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_2_2002_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_2_2003_40, PROFESSORES: [], VAGAS: 1.5
Nome do objeto: escola_3_1_40, PROFESSORES: [], VAGAS: 0.5
Nome do objeto: escola_3_2002_20, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_3_2003_20, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_4_1_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_4_2002_20, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_4_2003_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_5_1_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_5_2002_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_5_2003_40, PROFESSORES: [], VAGAS: 0.0
Nome do objeto: escola_6_1_40, PROFESSORES: [], VAGAS: 0.0
-----

In [630]:

# CRIANDO AS LISTAS PARA LOTAÇÃO TEMPORARIOS
for nome_objeto, escola in escolas_dict.items():

    for index, professor in base_professores.iterrows():

        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA'])and (escola.MUNICIPIO == professor['MUNICIPIO']):
            if professor['JORNADA'] == escola.CH_HORARIA:
                pass
            else:
                servidor = professor['SERVIDOR']
                
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)

        else:
            pass

for nome_objeto, escola in escolas_dict.items():

    for index, professor in base_professores.iterrows():

        if (escola.COD_DISCIPLINA == professor['COD_DISCIPLINA'])and (escola.MUNICIPIO == professor['MUNICIPIO']):
                
                servidor = professor['SERVIDOR']
                
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)

        else:
            pass


print('-'*20)
for objeto_print, escola_print in escolas_dict.items():
    print(f"Nome do objeto: {objeto_print}, PROFESSORES: {escola_print.lista_professores}, VAGAS: {escola_print.VAGAS}")
print('-'*20)

lotacao(base_professores) # DataFrame usado


--------------------
Nome do objeto: escola_1_1_40, PROFESSORES: ['Professor 30', 'Professor 29'], VAGAS: 0.0
Nome do objeto: escola_1_2002_40, PROFESSORES: ['Professor 10', 'Professor 9'], VAGAS: 0.5
Nome do objeto: escola_1_2003_20, PROFESSORES: ['Professor 19', 'Professor 20'], VAGAS: 0.0
Nome do objeto: escola_2_1_40, PROFESSORES: ['Professor 30', 'Professor 29'], VAGAS: 0.0
Nome do objeto: escola_2_2002_40, PROFESSORES: ['Professor 10', 'Professor 9'], VAGAS: 0.0
Nome do objeto: escola_2_2003_40, PROFESSORES: ['Professor 20', 'Professor 19'], VAGAS: 1.5
Nome do objeto: escola_3_1_40, PROFESSORES: ['Professor 30', 'Professor 29'], VAGAS: 0.5
Nome do objeto: escola_3_2002_20, PROFESSORES: ['Professor 9', 'Professor 10'], VAGAS: 0.0
Nome do objeto: escola_3_2003_20, PROFESSORES: ['Professor 19', 'Professor 20'], VAGAS: 0.0
Nome do objeto: escola_4_1_40, PROFESSORES: ['Professor 30', 'Professor 29'], VAGAS: 0.0
Nome do objeto: escola_4_2002_20, PROFESSORES: ['Professor 9', 'Professor 

In [631]:


vagas_aberto = pd.DataFrame(columns=['URE', 'USE', 'MUNICIPIO', 'AGRUPAMENTO', 'COD_ESCOLA', 'COD_MEC','COD_SETOR', 'ESCOLA','COD_DISCIPLINA', 'CH_HORARIA', 'VAGAS', 'CH_RELATIVA'])

for nome_objeto, escola in escolas_dict.items():
    # Verifica se a lista de professores não está vazia
    if escola.VAGAS > 0:
        # Criando um dicionário com os dados da escola e do professor
        dados = {
            'URE': escola.URE,
            'USE': escola.USE,
            'MUNICIPIO': escola.MUNICIPIO,
            'AGRUPAMENTO': escola.AGRUPAMENTO,
            'COD_ESCOLA': escola.COD_ESCOLA,
            'COD_MEC': escola.COD_MEC,
            'COD_SETOR': escola.COD_SETOR,
            'ESCOLA': escola.ESCOLA,
            'COD_DISCIPLINA': escola.COD_DISCIPLINA,
            'CH_HORARIA': escola.CH_HORARIA,
            'VAGAS': escola.VAGAS,
            'CH_RELATIVA': escola.CH_RELATIVA
        }

        # Adicionando os dados do professor ao DataFrame professores_lotados
        vagas_aberto = vagas_aberto._append(dados, ignore_index=True)

display(vagas_aberto)


,URE,USE,MUNICIPIO,AGRUPAMENTO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,CH_HORARIA,VAGAS,CH_RELATIVA


In [632]:
professores_lotados = professores_lotados.sort_values(by='COD_ESCOLA', ascending=True)

display(base_professores)


with pd.ExcelWriter('lotacaoNova.xlsx') as writer:
    professores_lotados.to_excel(writer, sheet_name='Lotação', index=False)
    base_professores.to_excel(writer, sheet_name='Não Lotados', index=False)
    vagas_aberto.to_excel(writer, sheet_name='Vagas em Aberto', index=False)


,URE,USE,MUNICIPIO,AGRUPAMENTO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,SERVIDOR,...,CH_SEMANAL,CH_TOTAL,area_conhecimento,TURNO_N_APTO,TURNO_CONCENTRADO,PREFERENCIA 1,PREFERENCIA 2,PREFERENCIA 3,PREFERENCIA 4,PREFERENCIA 5
8,01A URE - BRAGANCA,NaN,BRAGANCA,A,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,Professor 9,...,3,33,SIM,NAO,M,1,2,3,4.0,5.0
28,01A URE - BRAGANCA,NaN,BRAGANCA,A,4,15057291,1920913530000,EEEF PATALINO,1,Professor 29,...,3,33,SIM,N,T,1,2,3,4.0,5.0


PermissionError: [Errno 13] Permission denied: 'lotacaoNova.xlsx'